# Adding Confusion Species to Training Set


ANIA and AKEK were often false positives, this code is to get the 2021 XC snapshot recordings of ANIA and AKEK into the 5sec training set

In [32]:
import pandas as pd
from pathlib import Path
from glob import glob
from opensoundscape import CNN

Step 1: get the names of all audio files we will use

In [33]:
#gather ania XC mp3 names
ANIA_path = Path("/media/emu/datasets/annotated/xeno_canto_v2_1_2021/2021_12_04/magumma-parva/mp3s")
ANIA_recordings = glob(f"{ANIA_path}/*.mp3")

#gather akek XC mp3 names
AKEK_path = Path("/media/emu/datasets/annotated/xeno_canto_v2_1_2021/2021_12_04/loxops-caeruleirostris/mp3s")
AKEK_recordings = glob(f"{AKEK_path}/*.mp3")

#gather trainset mp3 names
KAAM_path_mac = Path("/home/dah238/Kauai-Amakihi/Audio/Macaulay Focal Recordings")
KAAM_path_XC = Path("/home/dah238/Kauai-Amakihi/Audio/Xeno-canto/mp3s")
KAAM_recordings = glob(f"{KAAM_path_mac}/*.wav") + glob(f"{KAAM_path_mac}/*.mp3") + glob(f"{KAAM_path_XC}/*")

fullaudionames = ANIA_recordings + AKEK_recordings + KAAM_recordings
type(fullaudionames)

list

step 2: create boxed annotations then make 5 second clip labels

In [ ]:
from opensoundscape import BoxedAnnotations
# Set the current directory to where the dataset is downloaded
dataset_path = Path("/home/dah238/Kauai-Amakihi/Annotations")

# Make a list of all of the csvs utilized
selection_files = glob(f"{dataset_path}/*/*.csv")
selection_files
train =pd.concat((pd.read_csv(selection_files[1]), pd.read_csv(selection_files[4])))
train['annotation'] = train['annotation'].replace([' KAAM_song','KAAM_wsong'],'KAAM_song')
train["annotation"].value_counts()

annotation
KAAM_song        323
KAAM_call         80
?                 20
?_KAAM_song       16
?_KAAM_call        7
?KAAM_song         6
?KAAM_call         1
  ?_KAAM_song      1
Name: count, dtype: int64

In [42]:
selection_files[3],selection_files[4]

('/home/dah238/Kauai-Amakihi/Annotations/akek_sam/sam_lapp_annotations.csv',
 '/home/dah238/Kauai-Amakihi/Annotations/xeno_canto/combined_output_xeno_canto.csv')

In [37]:
train_boxed = BoxedAnnotations(df = train, audio_files = fullaudionames)
class_sub = ["KAAM_song","?KAAM_song","?_KAAM_song","  ?_KAAM_song"]

train_clips = train_boxed.clip_labels(
    clip_duration = 5, 
    clip_overlap = 2.5,
    min_label_overlap = 0.25,
    audio_files = fullaudionames,
    class_subset = class_sub
)


In [38]:
train_clips = train_clips[~(train_clips["  ?_KAAM_song"] | train_clips["?KAAM_song"] | train_clips["?_KAAM_song"])]
train_clips_pure = train_clips[["KAAM_song"]]
train_clips_pure

KAAM_song
file                                               start_time end_time           
/media/emu/datasets/annotated/xeno_canto_v2_1_2... 0.0        5.0           False
                                                   2.5        7.5           False
                                                   5.0        10.0          False
                                                   7.5        12.5          False
                                                   10.0       15.0          False
...                                                                           ...
/home/dah238/Kauai-Amakihi/Audio/Xeno-canto/mp3... 25.0       30.0          False
                                                   27.5       32.5          False
                                                   30.0       35.0          False
                                                   32.5       37.5          False
                                                   35.0       40.0          False

[1574 rows x 1 columns]

In [65]:
xc_files = (
    pd.read_csv(
        "/media/emu/datasets/annotated/xeno_canto_v2_1_2021/labels.csv", index_col=0
    )
    .sample(3000)
    .index.values
)
xc_labels = pd.DataFrame(
    {"file": xc_files, "start_time": 0, "end_time": 5, "KAAM_song": False}
).set_index(["file", "start_time", "end_time"])
train_w_xc = pd.concat([train_clips_pure, xc_labels])

In [68]:
train_w_xc.KAAM_song.value_counts()

KAAM_song
False    3837
True      737
Name: count, dtype: int64

In [58]:
valid = pd.read_csv(selection_files[3])
valid_boxed = BoxedAnnotations(df = valid, audio_files=valid["audio_file"])
# Create dataframe of one-hot labels
class_sub = ["KAAM_song","KAAM_song?","AKEK_KAAM","KAAM_ANIA"]
valid["annotation"].unique()

array(['KAAM_song', '0', 'KAAM_song?', 'AKEK_song?', 'AKEK_KAAM',
       'KAAM_ANIA', 'ANIA_song?', 'ANIA_song', 'u', 'AKEK_song',
       'playback'], dtype=object)

In [59]:


valid_clips = valid_boxed.clip_labels(
    clip_duration = 3,
    clip_overlap = 0,
    min_label_overlap = 0,
    audio_files = valid["audio_file"].unique(), #CHANGE THIS
    class_subset = class_sub
)
ann = valid_boxed.df[["audio_file", "start_time", "end_time"]]

def overlaps_any(row):
    a = ann[ann.audio_file == row.name[0]]
    return ((a.start_time < row.name[1] + 3) &
            (a.end_time > row.name[1])).any()

valid_clips = valid_clips[valid_clips.apply(overlaps_any, axis=1)]

In [ ]:
valid_clips_pure = valid_clips[~(valid_clips["KAAM_song?"] | valid_clips["KAAM_ANIA"] | valid_clips["AKEK_KAAM"])]
valid_clips_pure = valid_clips_pure[["KAAM_song"]]
valid_clips_pure

KAAM_song
file                                               start_time end_time           
/media/kiwi/datasets/finalized/kaua2024a/AKEK_g... 93.0       96.0           True
                                                   147.0      150.0          True
/media/kiwi/datasets/finalized/kaua2024a/AKEK_g... 78.0       81.0           True
/media/kiwi/datasets/finalized/kaua2024a/AKEK_g... 102.0      105.0          True
                                                   108.0      111.0          True
...                                                                           ...
/media/kiwi/datasets/finalized/kaua2024a/PUAI_O... 150.0      153.0         False
                                                   165.0      168.0         False
                                                   171.0      174.0         False
/media/kiwi/datasets/finalized/kaua2024a/PUAI_O... 9.0        12.0          False
                                                   12.0       15.0          False

[419 rows x 1 columns]

save as csv and inspect

In [69]:
train_w_xc.to_csv("/home/dah238/Kauai-Amakihi/Transfer_Learning/Experiment5_s/train_set5_sec_no_?.csv")
valid_clips_pure.to_csv("/home/dah238/Kauai-Amakihi/Transfer_Learning/Experiment5_s/valid_set_no_?.csv")